# NBA Season

### Data Initialization

We are pulling the NBA season stats throughout the years from the URL of https://www.kaggle.com/datasets/justinas/nba-players-data/data

In [32]:
import pandas as pd

pd.__version__

'1.5.3'

**Get the kaggle dataset**

In [33]:
# read a csv file into a df
df = pd.read_csv('nba.csv')

dfTeams = pd.read_csv('unique_teams.csv')

df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


**Put abbreviations to Cities**

Some cities have multiple as franchises have changed

Cities had to be added as ESPN win rates used cities

In [34]:
# merge the two dataframes on team_abbreviation with df and dfTeams on abbreviations
df = pd.merge(df, dfTeams, left_on='team_abbreviation', right_on='abbreviations')

# drop the team_abbreviation column and abbreviations column
df = df.drop(columns=['team_abbreviation', 'abbreviations'])

In [35]:
# drop the unnamed column
df.drop('Unnamed: 0', axis=1, inplace=True)

df.head()

df.dtypes

player_name       object
age              float64
player_height    float64
player_weight    float64
college           object
country           object
draft_year        object
draft_round       object
draft_number      object
gp                 int64
pts              float64
reb              float64
ast              float64
net_rating       float64
oreb_pct         float64
dreb_pct         float64
usg_pct          float64
ts_pct           float64
ast_pct          float64
season            object
team              object
dtype: object

In [36]:
# look for null values
df.isnull().sum()

player_name      0
age              0
player_height    0
player_weight    0
college          0
country          0
draft_year       0
draft_round      0
draft_number     0
gp               0
pts              0
reb              0
ast              0
net_rating       0
oreb_pct         0
dreb_pct         0
usg_pct          0
ts_pct           0
ast_pct          0
season           0
team             0
dtype: int64

### Data Preprocessing

We need to deal with the columns we want to keep and also all the categorial data cols of:

player_name           object

team_abbreviation     object

college               object

country               object

draft_year            object

draft_round           object

draft_number          object

season                object

#### **Drop Some of Them**

In [37]:
df.drop(['player_name', 'college', 'draft_year', 'draft_round', 'draft_number'], axis=1, inplace=True)

#### **Country**

In [38]:
df['country'] = pd.Categorical(df['country']).codes

# print format the value counts with 'Country' as the header
print("Country value counts:\n\n%s" % df['country'].value_counts())

# Maybe drop this col for now
df.drop('country', axis=1, inplace=True)

Country value counts:

75    10569
12      203
25      189
2       100
63       93
      ...  
70        1
66        1
14        1
20        1
29        1
Name: country, Length: 82, dtype: int64


**Encode Season**

In [39]:
# categorical code season col but i want to keep the original
df['seasonEncoded'] = pd.Categorical(df['season']).codes

# print the unique values of season and seasonEncoded
unique_pairings = df[['season', 'seasonEncoded']].drop_duplicates()
print("Season unique pairings:\n", unique_pairings)

# drop the rows where seasonEncoded is less than 7. This keeps the season of 03-04 and later
df = df[df['seasonEncoded'] >= 7]

df.drop('season', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

Season unique pairings:
       season  seasonEncoded
0    1996-97              0
17   1997-98              1
31   1998-99              2
46   1999-00              3
62   2000-01              4
76   2001-02              5
90   2002-03              6
104  2003-04              7
119  2004-05              8
136  2005-06              9
152  2006-07             10
166  2007-08             11
183  2008-09             12
198  2009-10             13
214  2010-11             14
229  2011-12             15
244  2012-13             16
258  2013-14             17
274  2014-15             18
291  2015-16             19
306  2016-17             20
321  2017-18             21
340  2018-19             22
360  2019-20             23
380  2020-21             24
402  2021-22             25
418  2022-23             26


#### **Add in Win Rates**

In [40]:
import requests

def winRateFromYear(year, seasonEncoded):
    print(f'Getting data from {year}...{seasonEncoded}')
    # read the data into a df
    print(f'https://www.espn.com/nba/stats/rpi/_/year/{year}')

    # print if i have connection to url
    print(requests.get(f'https://www.espn.com/nba/stats/rpi/_/year/{year}').text)

    winRateDf = pd.read_html(f'https://www.espn.com/nba/stats/rpi/_/year/{year}')[0]

    # make the first row the header
    winRateDf.columns = winRateDf.iloc[1]

    #drop the first two rows
    winRateDf.drop([0, 1], inplace=True)

    winRateDf['PT DIFF'] = winRateDf['PF'].astype(float) - winRateDf['PA'].astype(float)

    winRateDf['seasonEncoded'] = seasonEncoded

    winRateDf = winRateDf[['TEAM', 'RPI', 'PCT', 'PT DIFF', 'seasonEncoded']]

    return winRateDf

**Merge DF2 with df on Team Names**

In [41]:
import time

# merge the two dataframes on team with df and teams on team where seasonEncoded is 7
winRateDf = pd.DataFrame()
seasonEncoded = 7

for year in range(2004, 2024):
    winRateDf = pd.concat([winRateDf, winRateFromYear(year, seasonEncoded)], ignore_index=True)
    time.sleep(4)
    seasonEncoded += 1

winRateDf.tail()

Getting data from 2004...7
https://www.espn.com/nba/stats/rpi/_/year/2004
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>403 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront)
Request ID: ojR2LHQPh1ZbxmfRREH0ZoZlwT5zglOZwnvFaTdpVpqtBCsBo63bOw==
</PRE>
<ADDRESS>
</ADDRESS>
</BODY></HTML>


HTTPError: HTTP Error 404: Not Found

**TODO: MERGE THE DFS**

In [12]:
merged_df = pd.merge(df, winRateDf, left_on=['team', 'seasonEncoded'], right_on=['TEAM', 'seasonEncoded'])

merged_df.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,team,seasonEncoded,TEAM,RPI,PCT,PT DIFF
0,40.0,205.74,111.130040,7,1.3,0.7,0.3,-7.9,0.000,0.217,0.165,0.521,0.100,Houston,7,Houston,.521,.549,142.0
1,33.0,200.66,122.469840,52,5.0,3.9,0.6,-5.0,0.104,0.175,0.152,0.538,0.063,Houston,7,Houston,.521,.549,142.0
2,28.0,193.04,97.522280,80,15.8,4.5,3.2,1.8,0.015,0.112,0.200,0.535,0.144,Houston,7,Houston,.521,.549,142.0
3,23.0,205.74,100.243832,45,3.1,1.6,0.7,2.0,0.021,0.136,0.159,0.477,0.103,Houston,7,Houston,.521,.549,142.0
4,29.0,195.58,104.326160,19,0.6,1.0,0.5,-8.4,0.010,0.167,0.093,0.278,0.132,Houston,7,Houston,.521,.549,142.0
